# This notebook contains the source code of the CS 521 class project. Specifically, this notebook contains the code for making an LLM roleplay the neighbor and landlord in the neighbor and landlord scenario respectively.

In [ ]:
# Connect to google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Install the required libraries
! pip -q install transformers
! pip install torchmetrics
! pip install evaluate
! pip install bert_score

Let's move to the desired folder in which we will store all our data.

In [ ]:
import os
os.chdir("/content/drive/MyDrive/RolePlay")

Try to chat with LLM without fine-tuning.

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer, AutoModelForCausalLM
from torchmetrics.text.rouge import ROUGEScore
from pprint import pprint
import torch
import re
# Replace TinyLlama/TinyLlama-1.1B-Chat-v1.0 with any other LLM trained with Causal LM objective
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
tokenizer.pad_token = '[PAD]'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Check if GPU is available
if torch.cuda.is_available():
    print("GPU is available.")
else:
  print("Not available")

GPU is available.


In [ ]:
# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens
    chat_history_ids = model.generate(
    bot_input_ids, max_length=1000,
    pad_token_id=tokenizer.eos_token_id
    )

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

![alt text](https://media.giphy.com/media/L3WevKXIKFDaZBvV8Q/giphy.gif)

Image from [Giphy](https://giphy.com/)

Not bad but not too impressive. We will fix it with fine-tuning.

## Model initial configuration

Let's train our own chatbot. For start, we will need basic configuration and a dataset.
Configuration and training scripts are mostly based on this [script](https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_language_modeling.py) from Huggingface and great [tutorial](https://nathancooper.io/i-am-a-nerd/chatbot/deep-learning/gpt2/2020/05/12/chatbot-part-1.html) from Nathan Cooper.

In [ ]:
"""
Fine-tuning the library models for language modeling on a text file (GPT, GPT-2, BERT, RoBERTa).
GPT and GPT-2 are fine-tuned using a causal language modeling (CLM) loss while BERT and RoBERTa are fine-tuned
using a masked language modeling (MLM) loss.
"""

import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple

import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange

from pathlib import Path

from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
    AutoModelForCausalLM
)


try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

# Configs
logger = logging.getLogger(__name__)

MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

# **Set the arguments here**

In [ ]:
# Args to allow for easy convertion of python script to notebook
class Args():
    def __init__(self):
        self.output_dir = '/content/drive/MyDrive/RolePlay/TinyLlama-1.1B-Chat-v1.0'  # set the output directory here
        self.model_type = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0' # set the model type here
        self.model_name_or_path = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0' # set the model id here
        self.config_name = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'
        self.tokenizer_name = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0' # set the tokenizer name here
        self.cache_dir = '/content/drive/MyDrive/RolePlay/cached' # set the path to cache directory here
        self.block_size = 512
        self.do_train = True
        self.do_eval = True
        self.evaluate_during_training = False
        self.per_gpu_train_batch_size = 2
        self.per_gpu_eval_batch_size = 2
        self.gradient_accumulation_steps = 1
        self.learning_rate = 5e-6
        self.weight_decay = 1e-2 # was 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 5 # was earlier 3
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 1000
        self.save_steps = 3500
        self.save_total_limit = 1
        self.eval_all_checkpoints = False
        self.no_cuda = False
        self.overwrite_output_dir = True
        self.overwrite_cache = True
        self.should_continue = False
        self.seed = 42
        self.local_rank = -1
        self.fp16 = False
        self.fp16_opt_level = 'O1'
        # Set paths to the folder where each dataset is stored.
        self.BD_Scene1 = '/content/drive/MyDrive/RolePlay/SSPA Data/SSPA Data/Complete transcript txt/BD_SC_1' # Bipolar Scene 1
        self.BD_Scene2 = '/content/drive/MyDrive/RolePlay/SSPA Data/SSPA Data/Complete transcript txt/BD_SC_2' # Bipolar Scene 2
        self.HC_Scene1 = '/content/drive/MyDrive/RolePlay/SSPA Data/SSPA Data/Complete transcript txt/HC_SC_1' # Healthy Control Scene 1
        self.HC_Scene2 = '/content/drive/MyDrive/RolePlay/SSPA Data/SSPA Data/Complete transcript txt/HC_SC_2' # Healthy Control Scene 2
        self.SZ_Scene1 = '/content/drive/MyDrive/RolePlay/SSPA Data/SSPA Data/Complete transcript txt/SZ_SC_1' # Schizophrenia Scene 1
        self.SZ_Scene2 = '/content/drive/MyDrive/RolePlay/SSPA Data/SSPA Data/Complete transcript txt/SZ_SC_2' # Schizophrenia Scene 2
        self.test_size = 0.15
        self.context_size = 800


args = Args()

## Prepare Dataset

In [ ]:
!pwd

/content/drive/MyDrive/RolePlay


Split our dataset into a training and test parts.

In [ ]:
# trn_df, val_df = train_test_split(df, test_size = 0.1)
# trn_df.head()

# This function will create a list of filenames (complete filename including their path) given the directory path
def list_files(directory):
    files = []  # List to store file names
    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)  # Get the full file path
        if os.path.isfile(filepath):  # Check if the path is a file
            files.append(filepath)  # Add the file path to the list
    return files


Comment the next 4 cells once the train and test data frames are prepared

In [ ]:
transcript_path = '/content/drive/MyDrive/RolePlay/SSPA Data/SSPA Data/Complete transcript txt/All transcripts'
BD_SC1_filenames = list_files(args.BD_Scene1)
BD_SC2_filenames = list_files(args.BD_Scene2)
HC_SC1_filenames = list_files(args.HC_Scene1)
HC_SC2_filenames = list_files(args.HC_Scene2)
SZ_SC1_filenames = list_files(args.SZ_Scene1)
SZ_SC2_filenames = list_files(args.SZ_Scene2)

In [ ]:
# trn_df and val_df are list of filenames(complete names with path)
BD_SC1_trn_df, BD_SC1_val_df = train_test_split(BD_SC1_filenames, test_size = args.test_size)
BD_SC2_trn_df, BD_SC2_val_df = train_test_split(BD_SC2_filenames, test_size = args.test_size)
HC_SC1_trn_df, HC_SC1_val_df = train_test_split(HC_SC1_filenames, test_size = args.test_size)
HC_SC2_trn_df, HC_SC2_val_df = train_test_split(HC_SC2_filenames, test_size = args.test_size)
SZ_SC1_trn_df, SZ_SC1_val_df = train_test_split(SZ_SC1_filenames, test_size = args.test_size)
SZ_SC2_trn_df, SZ_SC2_val_df = train_test_split(SZ_SC2_filenames, test_size = args.test_size)

In [ ]:
trn_df = BD_SC1_trn_df + BD_SC2_trn_df + HC_SC1_trn_df + HC_SC2_trn_df + SZ_SC1_trn_df + SZ_SC2_trn_df

In [ ]:
torch.save(trn_df,'/content/drive/MyDrive/RolePlay/trainset.pkl')

torch.save(BD_SC1_val_df, '/content/drive/MyDrive/RolePlay/BD_SC1_val.pkl')
torch.save(BD_SC2_val_df, '/content/drive/MyDrive/RolePlay/BD_SC2_val.pkl')
torch.save(HC_SC1_val_df, '/content/drive/MyDrive/RolePlay/HC_SC1_val.pkl')
torch.save(HC_SC2_val_df, '/content/drive/MyDrive/RolePlay/HC_SC2_val.pkl')
torch.save(SZ_SC1_val_df, '/content/drive/MyDrive/RolePlay/SZ_SC1_val.pkl')
torch.save(SZ_SC2_val_df, '/content/drive/MyDrive/RolePlay/SZ_SC2_val.pkl')

In [ ]:
# print(len(trn_df))
#print(len(val_df))

In [ ]:
# print(trn_df)

Now will convert our dataset in a format suitable for our model. Basically we will concatenate responses in one string for each conversation ending with the interviewer's response (additionally we will add special 'end of string' token between responses, so the model will understand end of each response in a string).  

In [ ]:
def constructConvDataset(filepath, prompt_string, tokenizer):
  datasetList=[] # This is a list of lists, where each list represents a part of conversation(in the form of token_ids)
                 # beginning with the prompt and ending with the interviewer's turn

  utterances = []  # This is a list of 2-tuples of (int, string), where the integer in the tuple represents if the
                   #string is of interviewer(1) or patient(0). Each string represents either patient's turn or the
                   # interviewer's turn
  current_utterance = ''  # Variable to store the current utterance
  current_label = None  # Variable to store the current label (0 for patient, 1 for interviewer)
  end_of_conversation = False  # Flag to indicate if the end cue is encountered

  # Open the text file for reading
  with open(filepath, 'r', encoding='utf-8') as file:
      # Iterate over each line in the file
      for line in file:
          #print("line: ", line)
          line = line.strip()  # Remove leading/trailing whitespaces
          if end_of_conversation:  # Stop parsing if the end cue is encountered
              break
          elif re.match(r'^#+.*', line):  # Check if the line contains '#' characters
              end_of_conversation = True  # Set the flag if the end cue is encountered
              continue
          elif line.startswith('Interviewer:'):
              if (current_utterance and (current_label == 0) ):  # If there's a previous utterance, store it
                  utterances.append((current_label, current_utterance))
                  current_utterance = line[len('Interviewer:'):].strip()  # Extract the utterance text
              elif(current_utterance and (current_label == 1)):
                current_utterance += ' ' + line[len('Interviewer:'):].strip()  # Extract the utterance text
              else:
                current_utterance = line[len('Interviewer:'):].strip()

              current_label = 1  # Set label to 1 for interviewer
          elif line.startswith('Patient:'):
              if (current_utterance and (current_label == 1) ) :  # If there's a previous utterance, store it
                  utterances.append((current_label, current_utterance))
                  current_utterance = line[len('Patient:'):].strip()
              elif(current_utterance and (current_label == 0)):
                current_utterance += ' ' + line[len('Patient:'):].strip()  # Extract the utterance text
              else:
                current_utterance = line[len('Patient:'):].strip()

              current_label = 0  # Set label to 0 for patient
          elif line.startswith('+++'):  # Skip lines with timestamps
              continue
          elif line and current_utterance:  # If the line is not empty and there is an ongoing utterance
              current_utterance += ' ' + line.strip()  # Concatenate with the previous line

  # Append the last utterance to the list
  if current_utterance:
      utterances.append((current_label, current_utterance))
  # print("Utterances: ", utterances)
  # print("filepath: ", filepath)

  start_index = 0
  end_index = 0
  if(utterances[0][0] == 0): # First utterance is of patient
    end_index = 1


  flatten = lambda somelist: [item for sublist in somelist for item in sublist]
  # print("start_index: ", start_index)
  # print("end_index: ", end_index)
  # print("len(utterances): ", len(utterances))
  # print("filepath: ", filepath)
  utterance_list = [tokenizer.encode(prompt_string)] + [[tokenizer.eos_token_id]] + [tokenizer.encode(utterances[x][1]) + [tokenizer.eos_token_id] for x in range(start_index,end_index+1)]
  utterance_list = flatten(utterance_list) #List of running utterance token_ids

  datasetList.append(utterance_list)


  end_index+=2
  while(end_index<len(utterances)):
    utterance_list = utterance_list + tokenizer.encode(utterances[end_index-1][1]) + [tokenizer.eos_token_id] + tokenizer.encode(utterances[end_index][1]) + [tokenizer.eos_token_id]
    newlen = len(utterance_list)
    if(newlen>args.context_size):

      while(start_index < end_index and newlen>args.context_size):

        start_index+=1
        if(start_index<end_index):
          utterance_list = tokenizer.encode(prompt_string) + [tokenizer.eos_token_id]
          for myind in range(start_index,end_index+1):
            utterance_list+=tokenizer.encode(utterances[myind][1]) +[tokenizer.eos_token_id]

          #utterance_list = [tokenizer.encode(prompt_string)] + [tokenizer.encode(utterances[x][1]) + [tokenizer.eos_token_id] for x in range(start_index,end_index+1)]

          #utterance_list = flatten(utterance_list)
          newlen = len(utterance_list)
        else:
          break


    datasetList.append(utterance_list)
    end_index+=2

  return datasetList

Set the Neighbor and Landlord prompts

In [ ]:
promptStringNeighbor = "I want you to roleplay a neighbor who has moved in to a new neighborhood. If your neighbor's utterance are related to the utterance you have already seen, please try to reuse the original response lines."
promptStringLandlord = "I want you to roleplay a landlord who is conversing with a tenant. If your tenant's utterance are related to the utterance you have already seen, please try to reuse the original response lines. The situation is that tenant had complained to you about a leak earlier but that has not been fixed yet."


In [ ]:
# Given a row(list of list of strings, where each string is a dialogue in a turn. The order of dialogues in the row is
# in reverse order, i.e. the most recent dialogue first, and the oldest dialogue last.), the function returns a single
# list where in which each element is the token id, and the individual dialogues are separated by the eos token. The
# order of dialogues in the returned list is oldest to newest, where the oldest dialogue is first and the most recent
# dialogue is last, and the last dialogue is the response.
def construct_conv(row, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
    conv = flatten(conv)
    return conv

# This class is used to create a custom dataset. Since args.overwrite_cache is true, the __init__ method overwrites the
# cached_features_file and creates a list of lists using the construct_conv function. The dataframe corresponding to the
# list of lists is a parameter of the init method. df is a list of strings, where each string
# represents a filename.
class ConversationDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, df, block_size=512):

        block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)
        print("Effective block size: ", block_size)

        directory = args.cache_dir
        cached_features_file = os.path.join(
            directory, args.model_type + "_cached_lm_" + str(block_size)
        )

        if os.path.exists(cached_features_file) and not args.overwrite_cache:
            logger.info("Loading features from cached file %s", cached_features_file)
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            logger.info("Creating features from dataset file at %s", directory)

            self.examples = [] # List of lists, where each list is a part of conversation
                               # represented as token ids

            # for _, row in df.iterrows():
            #     conv = construct_conv(row, tokenizer)
            #     self.examples.append(conv)

            for fil in df:
              if("Scene 1" in fil):
                #print(fil)
                conv_list = constructConvDataset(fil, promptStringNeighbor, tokenizer)
                self.examples += conv_list
              else:
                conv_list = constructConvDataset(fil, promptStringLandlord, tokenizer)
                self.examples += conv_list

            logger.info("Saving features into cached file %s", cached_features_file)
            #Check if directory structure exists before using pickle.dump
            cached_directory = os.path.dirname(cached_features_file)
            if not os.path.exists(cached_directory):
                os.makedirs(cached_directory)
            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)

In [ ]:
# Cacheing and storing of data/checkpoints

def load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False):
    return ConversationDataset(tokenizer, args, df_val if evaluate else df_trn)


def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

# This function returns sorted checkpoints in the directory given by args.output_dir. The checkpoints are either sorted
# by their modification time or by the number in the checkpoint file name. For example, if the checkpoints are
# "checkpoint-1", and "checkpoint-2", then the function will return a list of checkpoint file names:
# ["checkpoint-1", "checkpoint-2"]

def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
    ordering_and_checkpoint_path = []

    glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))

    for path in glob_checkpoints:
        if use_mtime:
            ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
        else:
            regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
            if regex_match and regex_match.groups():
                ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))

    checkpoints_sorted = sorted(ordering_and_checkpoint_path)
    checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
    return checkpoints_sorted

# This method deletes checkpoints if the number of checkpoints in the directory given by args.output_dir
# are greater than the number given by args.save_total_limit
def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
    if not args.save_total_limit:
        return
    if args.save_total_limit <= 0:
        return

    # Check if we should delete older checkpoint(s)
    checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
    if len(checkpoints_sorted) <= args.save_total_limit:
        return

    number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
    checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
    for checkpoint in checkpoints_to_be_deleted:
        logger.info("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
        shutil.rmtree(checkpoint)

# Define evaluation metrics



In [ ]:
from transformers import AutoModel

In [ ]:
#goldLabels, and generatedoutputs are lists of token ids that are passed during the evaluate method
def calculate_cosine(goldLabels, generatedoutputs, decoderTokenizer):
  device = torch.device("cuda")
  goldLabelsDecoded = list(map(lambda l: decoderTokenizer.decode(l,skip_special_tokens = True), goldLabels ))
  goldLabelsDecoded = torch.tensor(goldLabelsDecoded)
  goldLabelsDecoded = goldLabelsDecoded.to(device)

  generatedDecoded =  list(map(lambda l: decoderTokenizer.decode(l,skip_special_tokens = True), generatedoutputs ))
  generatedDecoded = torch.tensor(generatedDecoded)
  generatedDecoded = generatedDecoded.to(device)


  debertaTokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")
  debertaModel = AutoModel.from_pretrained("microsoft/deberta-base")
  debertaModel.to(device)

  goldInputs = debertaTokenizer(goldLabelsDecoded, return_tensors="pt",padding="longest")
  generatedInputs = debertaTokenizer(generatedDecoded, return_tensors="pt",padding="longest")

  debertaGoldOutputs = debertaModel(**goldInputs)
  debertaGeneratedOutput = debertaModel(**generatedInputs)

  goldOutputHiddenState = debertaGoldOutputs.last_hidden_state
  generatedOutputHiddenState = debertaGeneratedOutput.last_hidden_state

  goldOutputEmbedding = goldOutputHiddenState[:,0,:]
  generatedOutputEmbedding = generatedOutputHiddenState[:,0,:]

  return torch.mean(torch.nn.functional.cosine_similarity(goldOutputEmbedding, generatedOutputEmbedding))

In [ ]:
#goldLabels, and generatedoutputs are lists of token ids that are passed during the evaluate method
def calculate_rouge(goldLabels, generatedoutputs, decoderTokenizer):
  goldLabelsdecoded = list(map(lambda l: decoderTokenizer.decode(l,skip_special_tokens = True), goldLabels ))
  generatedDecoded =  list(map(lambda l: decoderTokenizer.decode(l,skip_special_tokens = True), generatedoutputs ))

  # print(goldLabelsdecoded[0:3])
  # print(generatedDecoded[0:3])
  rouge = ROUGEScore(rouge_keys=('rouge1', 'rougeL'))

  return rouge(generatedDecoded, goldLabelsdecoded)

In [ ]:
def calculate_bert_score(goldLabels, generatedoutputs, decoderTokenizer):
  goldLabelsdecoded = list(map(lambda l: decoderTokenizer.decode(l,skip_special_tokens = True), goldLabels ))
  generatedDecoded =  list(map(lambda l: decoderTokenizer.decode(l,skip_special_tokens = True), generatedoutputs ))

  print("goldLabelsdecoded: ", goldLabelsdecoded)
  print("generatedDecoded: ", generatedDecoded)
  from evaluate import load
  bertscore = load("bertscore")

  glempCount=0
  for l in goldLabelsdecoded:
    if(len(l) == 0):
      glempCount+=1

  print("Empty target sentence count: ", glempCount)

  # fulCount=0
  # for l in goldLabelsdecoded:
  #   if(len(l)>0):
  #     print("Printing string: ", l)
  #     fulCount+=1

  # print("Full target sentences: ", fulCount)

  glempCountPred=0
  for l in generatedDecoded:
    if(len(l) == 0):
      glempCountPred+=1

  print("Empty predicted sentence count: ", glempCountPred)

  # fulCountPred=0
  # for l in generatedDecoded:
  #   if(len(l)>0):
  #     print("Printing string: ", l)
  #     fulCountPred+=1

  # print("Full predicted sentences: ", fulCountPred)


  results = bertscore.compute(predictions=generatedDecoded, references=goldLabelsdecoded, model_type="microsoft/deberta-base")
  for k in results:
    # print(type(results[k][0]))
    # print(results[k])
    if(k!='hashcode'):
      results[k] = sum(results[k])/len(results[k])
  return results

# **Train & Evaluate**

In [ ]:
def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    # Should use tokenizer's padding mechanism instead of own.
    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate, drop_last = True
    )

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model.resize_token_embeddings(len(tokenizer))
    # add_special_tokens_(model, tokenizer)


    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            # labels = []

            # for example in batch:
            #   example = reversed(example)
            #   current_example = [tokenizer.eos_token_id]
            #   ind = 1
            #   while(ind<len(example)):
            #     if(example[ind] == tokenizer.eos_token_id):
            #       break
            #     current_example.append(example[ind])
            #     ind+=1
            #   current_example+=[-100]*(len(example)-len(current_example))
            #   current_example = reversed(current_example)
            #   labels.append(current_example)

            # Initialize an empty list to store labels

            labels = []
            attention_masks = []

            #print("batch.size(): ", batch.size())
            # Iterate over each example in the batch
            for example in batch:
                # Reverse the example
                #print("example: ", example)
                example = torch.flip(example,[0])
                # Find the indices of eos_token_id in the reversed example
                eos_indices = [i for i, x in enumerate(example) if x == tokenizer.eos_token_id]
                # Check if there are at least two occurrences of eos_token_id
                    # Get the index of the second occurrence of eos_token_id
                idx0 = eos_indices[0]
                idx1 = eos_indices[1]
                # Create the current example list
                current_example = torch.cat([torch.full((idx0,),-100),example[idx0:idx1]])
                # Pad the current example list with -100
                current_example = torch.cat([current_example, torch.full((len(example) - len(current_example),),-100)] )
                attention_mask = torch.cat([torch.full((idx0,),0),torch.full((len(example) - idx0,),1)])

                #current_example = torch.cat([current_example, torch.full((len(example) - len(current_example),),-100)] )
                # Reverse the current example list
                current_example = torch.flip(current_example,[0])
                # print("len(example): ", len(example))
                # print("len(current_example): ", len(current_example))

                # Append the current example to labels
                labels.append(current_example)
                attention_masks.append(attention_mask)

            labels = torch.stack(labels)
            attention_masks = torch.stack(attention_masks)



            inputs = batch
            #inputs, labels = (batch, batch)
            if inputs.shape[1] > 1024: continue
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            attention_masks = attention_masks.to(args.device)
            model.train()

            #This is wrong, should only compute loss for the response turn, here loss is being computed for every turn
            outputs = model(inputs, labels=labels, attention_mask = attention_masks)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            #This if updates the parameters when the step number in the current batch is a multiple of gradient_accumulation_steps
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                # Logging when the current value of global_step is a multiple of args.logging_steps
                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    #This is the average loss per step
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    # Delete excess checkpoints
                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step

# Evaluation of some model.
# Need to change this to incorporate our evaluation metrics

def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, df_trn, df_val, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir # output-small

    eval_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate, drop_last = True
    )

    # multi-gpu evaluate
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    labels = []
    generated_output = []

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        # inputs, labels = (batch, batch)
        # inputs = inputs.to(args.device)
        # labels = labels.to(args.device)

        # with torch.no_grad():
        #     outputs = model(inputs, labels=labels)
        #     #print("Outputs: ", outputs)
        #     lm_loss = outputs[0]
        #     # print("lm_loss: ", lm_loss)
        #     # print("lm_loss.item(): ",lm_loss.item() )
        #     # print("lm_loss.mean().item(): ",lm_loss.mean().item() )

        #     #lm_loss.mean() meaningless, not needed
        #     eval_loss += lm_loss.mean().item()
        # nb_eval_steps += 1

        inputs = []
        attention_masks=[]

        for example in batch:
          #print("Printing raw example: ", tokenizer.decode(example))
          example = torch.flip(example,[0])
          # Find the indices of eos_token_id in the reversed example
          eos_indices = [i for i, x in enumerate(example) if x == tokenizer.eos_token_id]
          # Check if there are at least two occurrences of eos_token_id

          # Get the index of the second occurrence of eos_token_id
          idx0 = eos_indices[0]
          idx1 = eos_indices[1]
          # Create the current example list
          current_example = example[idx0+1:idx1]
          # Pad the current example list with -100
          remaining_example = example[idx1:]
          # Reverse the current example list
          current_example = torch.flip(current_example,[0])
          remaining_example = torch.flip(remaining_example,[0])
          attention_mask = torch.full((len(remaining_example),),1)

          #print("Printing extracted label: ", tokenizer.decode(current_example))
          if(len(current_example) ==0):
            print("Empty label detected. eos_indices: ", eos_indices," full example: ", tokenizer.decode(torch.flip(example,[0])))
          labels.append(current_example.tolist())
          inputs.append(remaining_example)
          attention_masks.append(attention_mask)

        inputs = pad_sequence(inputs, batch_first = True)
        attention_masks = pad_sequence(attention_masks, batch_first = True)
        # print(attention_masks)
        # print("Inputs: ", inputs)
        device = torch.device("cuda")
        inputs = inputs.to(device)
        # if(inputs.size()[-1]>1024):
        #   print(inputs)
        #print("Before: ", inputs.size())
        attention_masks = attention_masks.to(device)
        generated_outputs = model.generate(inputs, max_new_tokens = 50, attention_mask = attention_masks, pad_token_id=tokenizer.eos_token_id)
        # print("inputs[0:3]: ", inputs[0:3])
        # print("generated_outputs[0:3]: ", generated_outputs[0:3])
        #print("After: ", generated_outputs.size())
        #print("Before shortening: ", list(map(lambda l: tokenizer.decode(l, skip_special_tokens = True), generated_outputs )))
        generated_outputs = list(map(lambda l1,l2: l2[len(l1):], inputs, generated_outputs))
        #print("After shortening: ", list(map(lambda l: tokenizer.decode(l, skip_special_tokens = True), generated_outputs )))
        #generated_outputs = generated_outputs.tolist()
        generated_output = generated_output + generated_outputs




    # eval_loss = eval_loss / nb_eval_steps
    # perplexity = torch.exp(torch.tensor(eval_loss))

    # result = {"perplexity": perplexity}

    # cosine_sim = calculate_cosine(labels, generated_output, tokenizer)
    # print("Cosine calculated")

    # print("Length of labels: ", len(labels))
    # print("Length of generated_output", len(generated_output))
    # print("Printing labels: ", list(map(lambda l: tokenizer.decode(l, skip_special_tokens = True), labels )))
    # print("Printing generated_output: ", list(map(lambda l: tokenizer.decode(l, skip_special_tokens = True), generated_output )))
    rouge_value = calculate_rouge(labels, generated_output, tokenizer)
    print("Rouge calculated")
    bertscore_value = calculate_bert_score(labels, generated_output, tokenizer)
    print("Bertscore calculated")

    #result = {"cosine_similarity": cosine_sim, "rouge_score": rouge_value, "bertscore": bertscore_value}
    result = {"rouge_score": rouge_value, "bertscore": bertscore_value}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

In [ ]:
# device = torch.device("cuda")
# args.n_gpu = torch.cuda.device_count()
# args.device = device
# model.to(args.device)
# result = evaluate(args, model, tokenizer, trn_df, val_df, prefix="prefix")

In [ ]:
import time

# **Main**

In [ ]:
# Main runner

def main(df_trn, df_val):
    args = Args()

    if args.should_continue:
        sorted_checkpoints = _sorted_checkpoints(args)
        if len(sorted_checkpoints) == 0:
            raise ValueError("Used --should_continue but no checkpoint was found in --output_dir.")
        else:
            args.model_name_or_path = sorted_checkpoints[-1]

    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
        and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Setup CUDA, GPU & distributed training
    device = torch.device("cuda")
    args.n_gpu = torch.cuda.device_count()
    args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

    # Set seed
    set_seed(args)

    config = AutoConfig.from_pretrained(args.config_name)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name)
    model = AutoModelForCausalLM.from_pretrained(
        args.model_name_or_path,
        from_tf=False,
        config=config,
    )
    model.to(args.device)

    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False)

        t1 = time.time()
        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        t2 = time.time()
        os.makedirs(os.path.join(args.output_dir, 'results'), exist_ok = True)
        loggingfilepath = os.path.join(args.output_dir, "results/time_results.txt")
        with open(loggingfilepath, "w") as file:
          # Write some text to the file
          file.write("Training time: " + str(t2-t1))
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()
    if args.do_train:
        # Create output directory if needed
        os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        model = AutoModelForCausalLM.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
        model.to(args.device)



In [ ]:
def evaluation_caller(df_val, df_trn, setting):
    device = torch.device("cuda")
    args.n_gpu = torch.cuda.device_count()
    args.device = device
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        t1 = time.time()
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = AutoModelForCausalLM.from_pretrained(checkpoint)
            model.to(args.device)
            tokenizer = AutoTokenizer.from_pretrained(checkpoint)
            #tokenizer.to(args.device)
            result = evaluate(args, model, tokenizer, df_trn, df_val, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

        t2 = time.time()
        os.makedirs(os.path.join(args.output_dir, 'results'), exist_ok = True)
        logpath = os.path.join(args.output_dir,'results/evaluation_time.txt')
        with open(logpath, "a") as file:
          # Write some text to the file
          file.write("\nEvaluation time of " + setting + ": " + str(t2-t1))

        respath = os.path.join(args.output_dir,'results/evaluation_results.txt')
        with open(respath, "a") as file:
          # Write some text to the file
          file.write("Evaluation results of: " + setting + ": " + str(results))

    return results

It is time to train our model!

![alt text](https://media.giphy.com/media/Tia3dkakIp2m4uGoDI/giphy.gif)

Image from [Giphy](https://giphy.com/)

In [ ]:
trn_df = torch.load('/content/drive/MyDrive/RolePlay/trainset.pkl')

BD_SC1_val_df = torch.load('/content/drive/MyDrive/RolePlay/BD_SC1_val.pkl')
BD_SC2_val_df = torch.load('/content/drive/MyDrive/RolePlay/BD_SC2_val.pkl')
HC_SC1_val_df = torch.load('/content/drive/MyDrive/RolePlay/HC_SC1_val.pkl')
HC_SC2_val_df = torch.load('/content/drive/MyDrive/RolePlay/HC_SC2_val.pkl')
SZ_SC1_val_df = torch.load('/content/drive/MyDrive/RolePlay/SZ_SC1_val.pkl')
SZ_SC2_val_df = torch.load('/content/drive/MyDrive/RolePlay/SZ_SC2_val.pkl')

In [ ]:
#Call main to train model
main(trn_df, BD_SC1_val_df)

In [ ]:
print("Final result of BD Scene1: ", evaluation_caller(BD_SC1_val_df, trn_df, "BD Scene 1"))
print("Final result of BD Scene2: ", evaluation_caller(BD_SC2_val_df, trn_df, "BD Scene 2"))
print("Final result of HC Scene1: ", evaluation_caller(HC_SC1_val_df, trn_df, "HC Scene 1"))
print("Final result of HC Scene2: ", evaluation_caller(HC_SC2_val_df, trn_df, "HC Scene 2"))
print("Final result of SZ Scene1: ", evaluation_caller(SZ_SC1_val_df, trn_df, "SZ Scene 1"))
print("Final result of SZ Scene2: ", evaluation_caller(SZ_SC2_val_df, trn_df, "SZ Scene 2"))

## Chatting with  LLM assistant

The model is ready, so it's time to chat with it.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
model = AutoModelForCausalLM.from_pretrained('output-small')

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    # print(new_user_input_ids)

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens,
    chat_history_ids = model.generate(
        bot_input_ids, max_length=200,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,
        do_sample=True,
        top_k=100,
        top_p=0.7,
        temperature = 0.8
    )

    # pretty print last ouput tokens from bot
    print("Bot: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))